# Week4: Building an Amharic E-commerce Data Extractor

## Task 1 Preprocessing the Scraped dataset

### Importing Required libraries

In [1]:
! pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 9.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import re
import nltk
import emoji
import random
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from unicodedata import normalize as uni_normalize

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Getting the Dataset from Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
columns = [
    'Channel Title',
    'Channel Username',
    'Message ID',
    'Message Text',
    'Date',
    'Media Path'
]

file_path = "/content/drive/MyDrive/kaim 6/week4/telegram_data.csv"

df = pd.read_csv(file_path, names=columns)

In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20055 entries, 0 to 20054
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Channel Title     20055 non-null  object 
 1   Channel Username  20055 non-null  object 
 2   Message ID        20055 non-null  int64  
 3   Message Text      11457 non-null  object 
 4   Date              20055 non-null  object 
 5   Media Path        0 non-null      float64
dtypes: float64(1), int64(1), object(4)
memory usage: 940.2+ KB
None


In [7]:
# Display the first 5 rows of the dataframe
print(df.head())

    Channel Title Channel Username  Message ID  \
0  Zemen Express®    @ZemenExpress        6994   
1  Zemen Express®    @ZemenExpress        6993   
2  Zemen Express®    @ZemenExpress        6992   
3  Zemen Express®    @ZemenExpress        6991   
4  Zemen Express®    @ZemenExpress        6990   

                                        Message Text  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3  💥💥...................................💥💥\n\n📌Sa...   
4                                                NaN   

                        Date  Media Path  
0  2025-06-21 16:35:51+00:00         NaN  
1  2025-06-21 16:35:51+00:00         NaN  
2  2025-06-21 16:35:51+00:00         NaN  
3  2025-06-21 16:35:51+00:00         NaN  
4  2025-06-21 08:07:31+00:00         NaN  


In [8]:
# Check the shape of the dataframe (number of rows and columns)
print("DataFrame shape:", df.shape)

# Get descriptive statistics for numerical columns (if any)
print(df.describe())

DataFrame shape: (20055, 6)
         Message ID  Media Path
count  20055.000000         0.0
mean    6985.697632         NaN
std     3914.146810         NaN
min        1.000000         NaN
25%     3921.500000         NaN
50%     6733.000000         NaN
75%    10113.000000         NaN
max    16437.000000         NaN


In [9]:
# Get the value counts for categorical columns (e.g., 'Channel Title')
# You can replace 'Channel Title' with other relevant categorical columns
if 'Channel Title' in df.columns:
  print("\nValue counts for 'Channel Title':")
  print(df['Channel Title'].value_counts())

# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())


Value counts for 'Channel Title':
Channel Title
Zemen Express®                           1000
NEVA COMPUTER®                           1000
EthioBrand®                              1000
ልዩ እቃ                                    1000
SINA KIDS/ሲና ኪድስⓇ                        1000
ⒶⓇⒶⒹⒶ ⒷⓇⒶⓃⒹ                              1000
Shewa Brand                              1000
HellooMarket                             1000
ሞደርን ሾፒንግ ሴንተር MODERN SHOPPING CENTER    1000
qnash.com - ቅናሽ ®️                       1000
Fashion tera                             1000
አዳማ ገበያ - Adama gebeya                   1000
ምርጥ ዕቃ                                   1000
AwasMart-አዋስማርት🎁                         1000
4Free Market🇪🇹                           1000
CLASSY BRAND ®                           1000
ማራኪ ცЯﾑŋの™                               1000
🇪🇹 📡 MARAKI BRANDS🌍🏆                     1000
መነሻዬ                                      930
BELLA CLASSIC®                            635
ኩሩ/kuru/ 🇪🇹 wear ®️            

In [10]:
def random_df():
    # Analyze the 'Message Text' column
    # Print some example message texts with better formatting
    print("--- Random Sample of Messages ---")
    sample_indices = random.sample(range(len(df)), 5)
    for i in sample_indices:
      print(f"Index: {i}")
      print(f"  Channel Username: {df['Channel Username'].iloc[i]}")
      print(f"  Message Text: {df['Message Text'].iloc[i]}")
      print("-" * 25)

random_df()


--- Random Sample of Messages ---
Index: 10394
  Channel Username: @Fashiontera
  Message Text: 〰️〰️〰️〰️〰️〰️〰️
CHEKICH
Made in Turkey
Size 40,41,42,43
Prices 3900
(Free Delivery)
Inbox @Hiwe5266
ስልክ +251945355266

    ♦️ ፋሽን ተራ /Fashion Tera♦️
          〰️〰️〰️〰️〰️〰️〰️〰️
አድራሻ: አዲስ አበባ ,ጦር ሀይሎች ድሪም ታወር 2ተኛ ፎቅ
ቢሮ ቁጥር 205
〰️〰️〰️〰️〰️
-------------------------
Index: 5699
  Channel Username: @sinayelj
  Message Text: nan
-------------------------
Index: 1573
  Channel Username: @nevacomputer
  Message Text: nan
-------------------------
Index: 16859
  Channel Username: @aradabrand2
  Message Text: nan
-------------------------
Index: 5180
  Channel Username: @sinayelj
  Message Text: nan
-------------------------


In [11]:
df['message_length'] = df['Message Text'].str.len()
print(f"\nAverage message length: {df['message_length'].mean():.2f}")


Average message length: 347.91


### Removing Rows that have empty text values

In [12]:
df.dropna(subset=['Message Text'], inplace=True)
print("\nMissing values after dropping NaNs in 'Message Text':")
print(df.isnull().sum())
print("\nDataFrame shape after dropping NaNs:", df.shape)


Missing values after dropping NaNs in 'Message Text':
Channel Title           0
Channel Username        0
Message ID              0
Message Text            0
Date                    0
Media Path          11457
message_length          0
dtype: int64

DataFrame shape after dropping NaNs: (11457, 7)


In [13]:
df.head()

,Channel Title,Channel Username,Message ID,Message Text,Date,Media Path,message_length
3,Zemen Express®,@ZemenExpress,6991,💥💥...................................💥💥\n\n📌Sa...,2025-06-21 16:35:51+00:00,NaN,450.0
7,Zemen Express®,@ZemenExpress,6987,💥💥...................................💥💥\n\n3pc...,2025-06-21 08:07:31+00:00,NaN,452.0
8,Zemen Express®,@ZemenExpress,6986,💥💥...................................💥💥\n\n3pc...,2025-06-21 08:07:11+00:00,NaN,452.0
9,Zemen Express®,@ZemenExpress,6985,💥💥...................................💥💥\n\n📌1 ...,2025-06-21 05:42:46+00:00,NaN,380.0
11,Zemen Express®,@ZemenExpress,6983,💥💥...................................💥💥\n\n📌1 ...,2025-06-21 05:42:19+00:00,NaN,380.0


### Tokenizing, Normalizing and Cleaning Text data

In [16]:
!pip install langdetect
!pip install spacy
!pip install transformers

In [39]:
import re
import nltk
import unicodedata
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
from nltk.tokenize import word_tokenize
import spacy
from transformers import AutoTokenizer

In [41]:
# --- Download necessary NLTK data ---
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
    nltk.download('punkt_tab')

# --- Load spaCy English model ---
try:
    nlp_en = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading spaCy 'en_core_web_sm' model...")
    spacy.cli.download("en_core_web_sm")
    nlp_en = spacy.load("en_core_web_sm")

# --- Load Hugging Face Amharic tokenizer ---
try:
    amharic_tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-r-base-amharic")
    print("Using 'Davlan/xlm-r-base-amharic' tokenizer.")
except Exception:
    print("Fallback: using 'xlm-roberta-base' tokenizer.")
    amharic_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# --- Language detection consistency ---
DetectorFactory.seed = 0

# --- Amharic normalization rules ---
AMHARIC_NORMALIZATION_MAP = {
    'ሃ': 'ሀ', 'ኅ': 'ሀ', 'ሐ': 'ሀ', 'ሓ': 'ሀ',
    'ሑ': 'ሁ', 'ኁ': 'ሁ', 'ዅ': 'ሁ',
    'ኂ': 'ሂ', 'ሒ': 'ሂ',
    'ኌ': 'ሄ', 'ሔ': 'ሄ',
    'ሕ': 'ህ',
    'ኆ': 'ሆ', 'ሖ': 'ሆ',
    'ሠ': 'ሰ',
    'ዐ': 'አ', 'ዓ': 'አ',
    'ጸ': 'ፀ',
    'ሷ': 'ሳ', 'ቷ': 'ታ', 'ኋ': 'ሃ'
}

def amharic_normalize(text):
    for variant, canon in AMHARIC_NORMALIZATION_MAP.items():
        text = text.replace(variant, canon)
    return text

def remove_emojis(text):
    # Unicode emoji ranges
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport
        "\U0001F1E0-\U0001F1FF"  # flags
        "\U00002700-\U000027BF"  # dingbats
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub("", text)

def preprocess_and_tokenize(text: str):
    """
    Cleans, normalizes, detects language, and tokenizes Amharic and English text.
    Emojis are removed completely.
    """
    text = text.lower()

    # Remove emojis
    text = remove_emojis(text)

    # Remove URLs, mentions, hashtags
    text = re.sub(r'http\S+|www\S+|https\S+', ' ', text)
    text = re.sub(r'@\w+', ' ', text)
    text = re.sub(r'#\w+', ' ', text)

    # Normalize Unicode
    text = unicodedata.normalize('NFKC', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Detect language
    try:
        detected_lang = detect(text)
    except LangDetectException:
        detected_lang = 'unknown'

    tokens = []
    if detected_lang == 'en':
        text = re.sub(r"[^\w\s']", ' ', text)
        doc = nlp_en(text)
        tokens = [token.text for token in doc]

    elif detected_lang == 'am':
        text = amharic_normalize(text)
        amharic_punct = "፡።፣፤፥፧፨፠"
        text = re.sub(f"[{re.escape(amharic_punct)}]", " ", text)
        text = re.sub(r"[^\u1200-\u137F0-9\s]", ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        tokens = amharic_tokenizer.tokenize(text)

    else:
        text = re.sub(r"[^\w\s]", ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        tokens = text.split()

    return detected_lang, tokens


Fallback: using 'xlm-roberta-base' tokenizer.


In [42]:
telegram_messages = random.sample([i for i in df['Message Text']], 5)
print("\nRandom Sample of 5 Message Texts:")
for i, msg in enumerate(telegram_messages):
    print(f"\n--- Processing Message {i+1} ---")
    print(f"Original: {msg}")
    lang, tokens = preprocess_and_tokenize(msg)
    print(f"Detected Language: {lang}")
    print(f"Tokens: {tokens}")


Random Sample of 5 Message Texts:

--- Processing Message 1 ---
Original: 🔥📣 ኪችንዎን በቅጽበት ውብ ማድረግ ይፈልጋሉ?

💁‍♀ለኪችንዎ ግርማ ሞገስ✨
✔️' ሀርቪ ስቲከር '🔼
☄️ ለመሳቢያ ፣ ለካቢኔ ፣ ለኩሽና ፣ ለስላሳ የእንጨት ገጽና ግድግዳ ላይ የሚለጠፍ ማስጌጫ
☄️ በራሱ የሚጣበቅ📎
ውሃ የማያሳልፍ (Waterproof) 
☄️ ሙቀትን🔥 እና እርጥበትን 🚿የሚቋቋም
☄️ ለማፅዳት ቀላል የሆነ
✨✨አንጸባራቂ✨✨

⚡️ Kitchen Sticker
Telegram - t.me/qnashcom
Size: 60CM×5M 👉
📌Water-proof 
📌Oil-proof 
📌Hot-proof

            💲 ዋጋ፦  800 ብር 

ውስን ፍሬ ነው የቀረው ❌❌

Telegram - t.me/qnashcom
Website - www.qnash.com
🔼ጥራት ✔️ ዋስትና 🛍 ቅናሽ

አድራሻ ፦ 
🗺ቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 

🗺ቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 

❌❌ ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ  የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !

ስልክ:
📱0946966440
📱0992606060
📱0905464599
Detected Language: ko
Tokens: ['ኪችንዎን', 'በቅጽበት', 'ውብ', 'ማድረግ', 'ይፈልጋሉ', 'ለኪችንዎ', 'ግርማ', 'ሞገስ', 'ሀርቪ', 'ስቲከር', 'ለመሳቢያ', 'ለካቢኔ', 'ለኩሽና', 'ለስላሳ', 'የእንጨት', 'ገጽና', 'ግድግዳ', 'ላይ', 'የሚለጠፍ', 'ማስጌጫ', 'በራሱ', 'የሚጣበቅ', 'ውሃ', 'የማያሳልፍ', 'waterproof', 'ሙቀትን', 'እና', 'እርጥበትን', 'የሚቋቋም', 'ለማፅዳት', '

### Creating a Normalized column from the text column

In [20]:
df['normalized_text'] = df['Message Text'].apply(lambda x: ' '.join(preprocess_and_tokenize(x)[1]) if pd.notnull(x) else None)


DataFrame with 'normalized_text' column:
                                         Message Text  \
3   💥💥...................................💥💥\n\n📌Sa...   
7   💥💥...................................💥💥\n\n3pc...   
8   💥💥...................................💥💥\n\n3pc...   
9   💥💥...................................💥💥\n\n📌1 ...   
11  💥💥...................................💥💥\n\n📌1 ...   

                                      normalized_text  
3                                        saachi el...  
7   3pcs bottle stopper በማንኛውም ጠርሙስ ጫፍ የሚገጠም ለዘይት ...  
8   3pcs bottle stopper በማንኛውም ጠርሙስ ጫፍ የሚገጠም ለዘይት ...  
9                                        1 pairs s...  
11                                       1 pairs s...  


In [44]:
# Randomly select a sample row
sample_row = df.sample(n=1).iloc[0]

# Print the original and normalized text
original_text = sample_row['Message Text']
normalized_text = sample_row['normalized_text']

print("\n--- Random Sampled Row ---")
print(f"Original Text: {original_text}")
print(f"Normalized Text: {normalized_text}")
print("-" * 25)


--- Random Sampled Row ---
Original Text: ⚡️⚡️🥘Momcoc Non Stick Fraying Pan
☄️ ከፍተኛ ጥራት Orignal 

📌 ሞምኮክ የሴራሚክ መጥበሻ
📌 መያዝ ፈፅሞ አይታሰብም
📌 ወጥ የሆነ የሙቀት ስርጭት የሚሰጥ
📌 ወፍራም ማራኪ መጥበሻ

➡️ ባለ 24,26 እና 28cm ስፋት
Telegram - t.me/qnashcom 👉

☄️ጥራት ✔️ ዋስትና 🛍 ቅናሽ

📍አድራሻ ፦ 
🗺ቁጥር 1 :- መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 

🗺ቁጥር 2 :- ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ G07 

❌❌ ማሳሰብያ: ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ  የራሳችን ሎጎ መኖሩን እና 329 / G07 መሆኑ ያረጋግጡ !

ስልክ:
📱0946966440
📱0992606060
📱0905464599
Normalized Text: momcoc non stick fraying pan ከፍተኛ ጥራት orignal ሞምኮክ የሴራሚክ መጥበሻ መያዝ ፈፅሞ አይታሰብም ወጥ የሆነ የሙቀት ስርጭት የሚሰጥ ወፍራም ማራኪ መጥበሻ ባለ 24 26 እና 28cm ስፋት telegram t me qnashcom ጥራት ዋስትና ቅናሽ አድራሻ ቁጥር 1 መገናኛ ዘፍመሽ ግራንድ ሞል 3 ተኛ ፎቅ ቁጥር 329 ቁጥር 2 ጀሞ 1 ከለላ ህንፃ ግራውንድ ለይ g07 ማሳሰብያ ሱቃችን ሲመጡ ትክክለኛ የኛ ሱቅ መሆኑ ያረጋግጡ የራሳችን ሎጎ መኖሩን እና 329 g07 መሆኑ ያረጋግጡ ስልክ 0946966440 0992606060 0905464599
-------------------------


In [ ]:
import re
# Define expanded keyword sets
product_keywords = {w.lower() for w in {
    'ስልክ', 'ቴሌቭዥን', 'ማይክሮዌቭ', 'ኮምፒውተር', 'መሣሪያ',
    'ልብስ', 'ጫማ', 'ሰነድ', 'ጌጣጌጥ', 'ቡና', 'ውሃ', 'ወተት', 'መጽሐፍ',
    'መኪና', 'ካሜራ', 'ፋርመሲ', 'ሪሞት', 'remote', 'magic remote', 'keyboard',
    'mouse', 'qwerty keyboard', 'air mouse', 'android box', 'vietnam', "ፋይንደር"
}}

location_keywords = {w.lower() for w in {
    'አዲስ', 'አበባ', 'ቦሌ', 'ላፍቶ', 'ደሬዳዋ', 'አሸዋ', 'ሚና', 'ህንፃ',
    'ገርጂ', 'ፕላዛ', 'ሜክሲኮ', 'ኢምፔሪያል', 'ከሳሚ', 'ሀበሻ', 'ኮፊ',
    'ጉርድ', 'ሾላ', 'ሆሊ', 'ሲቲ', 'ሴንተር', 'አራዳ', 'ባህርዳር', 'ትያቢ',
    'አቃቢማን', 'አቢዮት', 'ሚካኤል', 'አማረኛ',"ህንፃ", "ቁ", "ጀሞ"
}}

price_keywords = {'ብር', 'ዋጋ', 'በ', 'birr', 'price', "ሺ"}

# Phone number pattern
phone_number_pattern = re.compile(r'^(?:\+?251|0)9\d{8}$')

# Price number pattern
number_pattern = re.compile(r'^\d{1,7}([.,]?\d{1,3})*$')

def label_tokens(tokens):
    labels = []
    prev_label = 'O'

    def assign_label(entity_type, prev_label):
        return 'I-' + entity_type if prev_label.endswith(entity_type) else 'B-' + entity_type

    for token in tokens:
        token_clean = token.lower().strip("፣።.,:;!?\"“”’'")  # Strip punctuation

        if phone_number_pattern.match(token_clean):
            label = assign_label('PHONE', prev_label)

        elif token_clean in price_keywords:
            label = assign_label('PRICE', prev_label)

        elif number_pattern.match(token_clean):
            label = assign_label('PRICE', prev_label)

        elif token_clean in location_keywords:
            label = assign_label('LOC', prev_label)

        elif token_clean in product_keywords:
            label = assign_label('PRODUCT', prev_label)

        else:
            label = 'O'

        labels.append(label)
        prev_label = label

    return labels


# Sample 5 rows
sampled_df = df[['Message Text', 'normalized_text']].sample(n=5)

output_file = "data/raw/labeled_data_from_df.conll"

with open(output_file, "w", encoding="utf-8") as f:
    for i, row in sampled_df.iterrows():
        message = row['normalized_text']
        print(f"\n--- Labeling Message {i+1} ---")
        print(f"Message: {message}")

        tokens = word_tokenize(message)
        labels = label_tokens(tokens)

        for token, label in zip(tokens, labels):
            print(f"{token}\t{label}")
            f.write(f"{token}\t{label}\n")
        f.write("\n")

print("\n✅ Simulated labeling complete for 5 messages.")
print(f"📝 Labeled data saved to '{output_file}' in CoNLL format.")



--- Labeling Message 14183 ---
Message: 

--- Labeling Message 6373 ---
Message: ሰላም ውድ ደበኞቻችን ኦሪጅናል denim sharpa jacket የሚባል ጃኬት አስገብተናል የሚገራርሙ ሱሪም ቲሸርትም አስገብተናል እና ስቶክ crux ቱታም አስገብተናል ይምጡ ይጐብኙን ቁጥሩ size l የቤት ቁጥር 109 እና 110 አድራሻ ድሬዳዋ አሸዋ ሚና ህንፃ 1ኛ ፎቅ እንገኛለን በ inbox አዋሩን 0987336458 0939395045 ይደውሉልን
ሰላም	O
ውድ	O
ደበኞቻችን	O
ኦሪጅናል	O
denim	O
sharpa	O
jacket	O
የሚባል	O
ጃኬት	O
አስገብተናል	O
የሚገራርሙ	O
ሱሪም	O
ቲሸርትም	O
አስገብተናል	O
እና	O
ስቶክ	O
crux	O
ቱታም	O
አስገብተናል	O
ይምጡ	O
ይጐብኙን	O
ቁጥሩ	O
size	O
l	O
የቤት	O
ቁጥር	O
109	B-PRICE
እና	O
110	B-PRICE
አድራሻ	O
ድሬዳዋ	O
አሸዋ	B-LOC
ሚና	I-LOC
ህንፃ	I-LOC
1ኛ	O
ፎቅ	O
እንገኛለን	O
በ	B-PRICE
inbox	O
አዋሩን	O
0987336458	B-PHONE
0939395045	I-PHONE
ይደውሉልን	O

--- Labeling Message 7250 ---
Message: ተንቀሳቅሽ እና በቀላሉ የሚገጣጠም የልብስ ማስጫ ለማዘዝ 0974312223 ይደውሉ ወይም ይጠቀሙ ይዘት 2 ግድግዳ ላይ የሚገጠሙ ብረቶች ብሎን እና 20 ሜትር ገመድ
ተንቀሳቅሽ	O
እና	O
በቀላሉ	O
የሚገጣጠም	O
የልብስ	O
ማስጫ	O
ለማዘዝ	O
0974312223	B-PHONE
ይደውሉ	O
ወይም	O
ይጠቀሙ	O
ይዘት	O
2	B-PRICE
ግድግዳ	O
ላይ	O
የሚገጠሙ	O
ብረቶች	O
ብሎን	O
እና	O
20	B-PRICE
ሜትር	O
ገመድ	O

--- Labeling Message 1479 ---
Messa